In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,confusion_matrix,precision_recall_fscore_support

from sklearn.linear_model import SGDRegressor

import torch
import torch.nn as nn

# Reproducibility
RANDOM_STATE=42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)

# Data Prep
data=load_breast_cancer()
X=data.data.astype(np.float64)
y=data.target.astype(np.int64)

# Map {0,1} -> {-1,+1} for ADALINE
y=np.where(y==0,-1,1)

X_train,X_test,y_train,y_test=train_test_split(
    X,y,test_size=0.25,random_state=RANDOM_STATE,stratify=y
)

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

# Helpers
def metrics_report(y_true,y_pred,pos_label=1,name="Model"):
    from sklearn.metrics import accuracy_score,confusion_matrix,precision_recall_fscore_support
    import numpy as np
    import matplotlib.pyplot as plt

    acc=accuracy_score(y_true,y_pred)
    cm=confusion_matrix(y_true,y_pred,labels=[1,-1])
    prec,rec,f1,_=precision_recall_fscore_support(
        y_true,y_pred,average="binary",pos_label=pos_label,zero_division=0
    )

    print(f"\n{name} — Test Metrics")
    print(f"Accuracy:{acc:.4f}")
    print(f"Precision:{prec:.4f} Recall:{rec:.4f} F1:{f1:.4f}")
    print("Confusion Matrix (rows=[+1,-1], cols=[+1,-1]):")
    print(cm)

    fig,ax=plt.subplots(figsize=(5,4))
    im=ax.imshow(cm,interpolation="nearest")
    plt.colorbar(im,ax=ax,fraction=0.046,pad=0.04)

    class_names=["+1","-1"]
    ax.set(
        xticks=np.arange(len(class_names)),
        yticks=np.arange(len(class_names)),
        xticklabels=class_names,
        yticklabels=class_names,
        xlabel="Predicted label",
        ylabel="True label",
        title=f"{name} — Confusion Matrix"
    )

    thresh=cm.max()/2.0 if cm.size else 0.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(
                j,i,format(cm[i,j],"d"),
                ha="center",va="center",
                color="white" if cm[i,j]>thresh else "black",
                fontsize=12
            )

    plt.tight_layout()
    plt.show()

    return {"accuracy":acc,"precision":prec,"recall":rec,"f1":f1,"cm":cm}


def plot_sse(histories,title):
    plt.figure(figsize=(7,5))
    for label,sse in histories.items():
        plt.plot(range(1,len(sse)+1),sse,label=str(label))
    plt.xlabel("Epoch")
    plt.ylabel("SSE")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()
# SECTION 2 — SEMI-MANUAL (PyTorch Autograd)
class AdalineTorch(nn.Module):
    def __init__(self, n_features):
        super().__init__()
        self.linear = nn.Linear(n_features, 1, bias=True)
        with torch.no_grad():
            self.linear.weight.uniform_(-0.01, 0.01)
            self.linear.bias.fill_(0.0)

    def forward(self, x):
        return self.linear(x)  # identity

def train_adaline_torch(Xtr, ytr, lr=0.01, epochs=100):
    X_t = torch.tensor(Xtr, dtype=torch.float32)
    y_t = torch.tensor(ytr.reshape(-1, 1), dtype=torch.float32)
    model = AdalineTorch(Xtr.shape[1])
    opt = torch.optim.SGD(model.parameters(), lr=lr)
    criterion = nn.MSELoss(reduction="sum")  # SSE
    sse_hist = []
    for _ in range(epochs):
        opt.zero_grad()
        out = model(X_t)
        loss = criterion(out, y_t)
        loss.backward()
        opt.step()
        sse_hist.append(float(loss.item()))
    return model, sse_hist

# CLO-1: Train + SSE vs Epochs (Torch)
torch_lr = 0.01
torch_epochs = 100
torch_model, torch_hist = train_adaline_torch(X_train, y_train, lr=torch_lr, epochs=torch_epochs)

plot_sse({"SSE": torch_hist},
         f"ADALINE (PyTorch Autograd) — SSE vs Epochs (η={torch_lr})")

# CLO-2: Metrics on Test (Torch)
with torch.no_grad():
    logits = torch_model(torch.tensor(X_test, dtype=torch.float32)).numpy().ravel()
y_pred_torch = np.where(logits >= 0.0, 1, -1)
torch_metrics = metrics_report(y_test, y_pred_torch, name=f"ADALINE PyTorch (η={torch_lr})")

# CLO-3: Learning Rate Experiments (Torch)
lrs=[0.001,0.01,0.1,1.0]

hist_torch = {}
for lr in lrs:
    mdl, sse_hist = train_adaline_torch(X_train, y_train, lr=lr, epochs=torch_epochs)
    hist_torch[f"η={lr}"] = sse_hist

plot_sse(hist_torch, "ADALINE (PyTorch) — SSE vs Epochs for Learning Rates")

